In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from lightgbm import LGBMClassifier

from functools import partial
from hyperopt import hp,fmin, tpe, Trials
from hyperopt.pyll.base import scope

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [2]:
X_TRAIN_DIR = "processed_data/glcm/X_train.pkl"
y_TRAIN_DIR = "processed_data/glcm/y_train.pkl"
X_TEST_DIR = "processed_data/glcm/X_test.pkl"
y_TEST_DIR = "processed_data/glcm/y_test.pkl"

X_train = pd.read_pickle(X_TRAIN_DIR)
y_train = pd.read_pickle(y_TRAIN_DIR)
X_test = pd.read_pickle(X_TEST_DIR)
y_test = pd.read_pickle(y_TEST_DIR)

In [3]:
sc = StandardScaler()
sc.fit(X_train)
sc.fit(X_test)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [4]:
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)
# class_labels = label_encoder.classes_
# d_train = lgb.Dataset(X_train, label = y_train)

# #lr = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
# params = {}

# params['learning_rate'] = 0.05
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'multiclass'
# params['metric'] = 'multi_logloss'
# params['max_depth'] = 10
# params['num_leaves'] = 100
# params['num_class'] = len(class_labels)

# clf = lgb.train(params, d_train, 200)

# y_pred_1 = clf.predict(X_test)

# y_pred = [class_labels[idx] for idx in np.argmax(y_pred_1, axis=1)]

# cm = confusion_matrix(y_test, y_pred)

# # Plot confusion matrix as a heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
# plt.xlabel("Predicted")
# plt.ylabel("True")
# plt.title("Confusion Matrix")
# plt.show()

# # Print classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.356496 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17085
[LightGBM] [Info] Number of data points in the train set: 4129, number of used features: 67
[LightGBM] [Info] Start training from score -0.700683
[LightGBM] [Info] Start training from score -1.058265
[LightGBM] [Info] Start training from score -1.920562
[LightGBM] [Info] Start training from score -4.588121
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

In [ ]:
def optimize(params, x, y):
    clf = LGBMClassifier(**params, random_state=123)
    accuracies = []

    
    clf.fit(X_train,y_train)
    preds =  clf.predict(X_test)
    fold_acc = recall_score(y_test,preds, average="macro")
    accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies)

#defining a set of values as hp for hyperparameters
param_space2 = {'boosting_type': hp.choice('boosting_type',['gbdt','dart','goss']),
                'subsample_for_bin': scope.int(hp.quniform('subsample_for_bin', 20000, 300000, 20000)),
                'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
                'reg_alpha': hp.choice('reg_alpha', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
                'reg_lambda': hp.choice('reg_lambda', [0, hp.loguniform('lambda_l2_positive', -16, 2)]),
                "max_depth" : scope.int(hp.quniform("max_depth",3, 20, 1)) ,
                "n_estimators": scope.int(hp.quniform("n_estimators",100,1500,1)),
                'learning_rate': hp.uniform("learning_rate",0.01,1),
                'num_leaves': scope.int(hp.quniform('num_leaves', 24, 80, 1)),
                'subsample': hp.uniform("subsample", 0.01, 1.0)}
                                            
#defiing optimization_fuction as partial and calling optimize within it
optimization_fuction2 = partial(optimize, x = np.array(X_train), y = np.array(y_train)) 
trials2 = Trials()

#Getting the optimum values for hyperparameters
result2 = fmin(
    fn = optimization_fuction2,
    space = param_space2,
    algo = tpe.suggest,
    max_evals = 15,
    trials = trials2,
)

#Printing the best hyperparemeter set
print(result2)


LightGBMError: Number of classes should be specified and greater than 1 for multiclass training